In [ ]:
import pandas as pd
import numpy as np
import os

# Folder where raw CSVs are stored
RAW_PATH = "/Users/erion/Desktop/MSCF2/ADA/Capstone-Project/data/raw/"

datasets = {}

# Load all raw CSV files
for file in os.listdir(RAW_PATH):
    if file.endswith(".csv"):
        name = file.replace(".csv", "")
        # Skip the first 3 rows (header rows), set Date as index
        df = pd.read_csv(RAW_PATH + file, skiprows=3, index_col=0, parse_dates=True)
        # Set column names based on the structure (Close, High, Low, Open, Volume)
        df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
        df.index.name = 'Date'
        datasets[name] = df

# Show which datasets we loaded
print("Loaded datasets:", list(datasets.keys()))
print("\nDataset shapes:")
for name, df in datasets.items():
    print(f"{name}: {df.shape}")


Loaded datasets: ['gold', 'eurusd', 'treasury_10y', 'spy', 'vix', 'dxy', 'oil']

Dataset shapes:
gold: (5524, 5)
eurusd: (5472, 5)
treasury_10y: (5530, 5)
spy: (5536, 5)
vix: (5536, 5)
dxy: (5549, 5)
oil: (5528, 5)


In [7]:
# Standardize all datasets: keep only 'Close' price
for name, df in datasets.items():
    # All datasets have a 'Close' column after loading
    datasets[name] = df[['Close']].rename(columns={'Close': name})

# Show preview of the cleaned datasets
for name, df in datasets.items():
    print(f"{name}: {df.shape}")
    display(df.head())


KeyError: "None of [Index(['Close'], dtype='object')] are in the [columns]"